In [13]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key
from config import api_key #old weather api key that is blocked
# Import the datetime module from the datetime library.
from datetime import datetime

#import time to add sleep in weather api for loop to avoid making more than 60 calls within a minute
import time

#traceback
import traceback

# Add str libraray to get join for weather description joing if multiple items are found for weather  in case we need it
from str import join

ModuleNotFoundError: No module named 'str'

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

749

In [5]:
#checking cities names
cities 

['norman wells',
 'bluff',
 'qaanaaq',
 'bagdarin',
 'waipawa',
 'hobart',
 'mys shmidta',
 'dunedin',
 'sao filipe',
 'khatanga',
 'georgetown',
 'punta arenas',
 'iwanai',
 'solone',
 'rikitea',
 'eureka',
 'mataura',
 'teul',
 'souillac',
 'atuona',
 'tsihombe',
 'zielona gora',
 'hilo',
 'grand river south east',
 'bambous virieux',
 'katsuura',
 'tatarbunary',
 'ushuaia',
 'lorengau',
 'bethel',
 'dikson',
 'warqla',
 'tondano',
 'riyadh',
 'grants pass',
 'hermanus',
 'port macquarie',
 'portland',
 'lavrentiya',
 'zheleznodorozhnyy',
 'dunmore town',
 'mount isa',
 'butaritari',
 'talnakh',
 'riberalta',
 'roald',
 'westport',
 'barentsburg',
 'hithadhoo',
 'port alfred',
 'mananjary',
 'belz',
 'saleaula',
 'sentyabrskiy',
 'victoria',
 'cape town',
 'samusu',
 'bredasdorp',
 'tabou',
 'havre-saint-pierre',
 'albany',
 'narsaq',
 'tiksi',
 'salalah',
 'thompson',
 'saint george',
 'raudeberg',
 'tuktoyaktuk',
 'saldanha',
 'arraial do cabo',
 'port elizabeth',
 'midland',
 'por

In [6]:
len(cities)

749

In [7]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
#url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" +  api_key

In [15]:
### ADDED CODE FOR TESTING AS OLD API KEY WAS LOKCED SO TRIED CREATING A NEW ONE USING NEW USER EMAIL ADDRESS
city="portland"
city_url = url + "&q=" + city.replace(" ","+")
#print(city_url)
city_weather = requests.get(city_url).json()
city_weather
#len(city_weather["weather"]) 
#Adding code to test how to concatnate weather description if more than one found
j=0
city_weather_desc_list = []
while j < len(city_weather["weather"]):
    city_weather_desc_list.append(city_weather["weather"][j]["description"])
    j +=1
city_weather_desc =  "," .join(city_weather_desc_list)
city_weather_desc

'overcast clouds'

In [19]:
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(25)  # add 25 sec delay for every set to avoid going over limit
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    ##print(city_url) # for validating
 

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    city_weather = requests.get(city_url).json()
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        #add weather description weather is a list  
        j=0
        city_weather_desc_list = []
        while j < len(city_weather["weather"]): #description is a list and can have more than one so for now picking firs item
            city_weather_desc_list.append(city_weather["weather"][j]["description"])
            j +=1
        city_weather_desc =  "," .join(city_weather_desc_list)
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Current Weather" : city_weather_desc})


# If an error is experienced, skip the city.
    except :
        print(f"City not found. Skipping city | {city}...")
        traceback.print_exc() ## added to troubleshoot the issue after adding weather description
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | norman wells
Processing Record 2 of Set 1 | bluff
Processing Record 3 of Set 1 | qaanaaq
Processing Record 4 of Set 1 | bagdarin
Processing Record 5 of Set 1 | waipawa
Processing Record 6 of Set 1 | hobart
Processing Record 7 of Set 1 | mys shmidta
City not found. Skipping city | mys shmidta...
Processing Record 8 of Set 1 | dunedin


Traceback (most recent call last):
  File "<ipython-input-19-67dc948eaab5>", line 33, in <module>
    city_lat = city_weather["coord"]["lat"]
KeyError: 'coord'


Processing Record 9 of Set 1 | sao filipe
Processing Record 10 of Set 1 | khatanga
Processing Record 11 of Set 1 | georgetown
Processing Record 12 of Set 1 | punta arenas
Processing Record 13 of Set 1 | iwanai
Processing Record 14 of Set 1 | solone
Processing Record 15 of Set 1 | rikitea
Processing Record 16 of Set 1 | eureka
Processing Record 17 of Set 1 | mataura
Processing Record 18 of Set 1 | teul
City not found. Skipping city | teul...
Processing Record 19 of Set 1 | souillac


Traceback (most recent call last):
  File "<ipython-input-19-67dc948eaab5>", line 33, in <module>
    city_lat = city_weather["coord"]["lat"]
KeyError: 'coord'


Processing Record 20 of Set 1 | atuona
Processing Record 21 of Set 1 | tsihombe
City not found. Skipping city | tsihombe...
Processing Record 22 of Set 1 | zielona gora


...

Processing Record 44 of Set 14 | tura
Processing Record 45 of Set 14 | peniche
Processing Record 46 of Set 14 | sur
Processing Record 47 of Set 14 | kuzino
Processing Record 48 of Set 14 | kadhan
Processing Record 49 of Set 14 | atasu
Processing Record 50 of Set 14 | zamora
Processing Record 1 of Set 15 | batticaloa
Processing Record 2 of Set 15 | balikpapan
Processing Record 3 of Set 15 | ritchie
Processing Record 4 of Set 15 | isiolo
Processing Record 5 of Set 15 | morehead
Processing Record 6 of Set 15 | lumsden
Processing Record 7 of Set 15 | samarinda
Processing Record 8 of Set 15 | natal
Processing Record 9 of Set 15 | yafran
Processing Record 10 of Set 15 | panama city
Processing Record 11 of Set 15 | bonavista
Processing Record 12 of Set 15 | lagoa
Processing Record 13 of Set 15 | pandan
Processing Record 14 of Set 15 | stargard szczecinski
Processing Record 15 of Set 15 | tahoua
Processing Record 16 of Set 15 | innisfail
Processing Record 17 of Set 15 | tchollire
Processing Re

Traceback (most recent call last):
  File "<ipython-input-19-67dc948eaab5>", line 33, in <module>
    city_lat = city_weather["coord"]["lat"]
KeyError: 'coord'


Processing Record 33 of Set 15 | batagay-alyta
Processing Record 34 of Set 15 | calabozo
Processing Record 35 of Set 15 | shasta lake
Processing Record 36 of Set 15 | los llanos de aridane
Processing Record 37 of Set 15 | lagunas
Processing Record 38 of Set 15 | braintree
Processing Record 39 of Set 15 | pecanha
Processing Record 40 of Set 15 | bundaberg
Processing Record 41 of Set 15 | tautira
Processing Record 42 of Set 15 | augusto correa
Processing Record 43 of Set 15 | perivolion
Processing Record 44 of Set 15 | sfantu gheorghe
Processing Record 45 of Set 15 | gondanglegi
Processing Record 46 of Set 15 | buala
Processing Record 47 of Set 15 | westpunt
City not found. Skipping city | westpunt...
Processing Record 48 of Set 15 | aksu
Processing Record 49 of Set 15 | spornoye

Traceback (most recent call last):
  File "<ipython-input-19-67dc948eaab5>", line 33, in <module>
    city_lat = city_weather["coord"]["lat"]
KeyError: 'coord'



-----------------------------
Data Retrieval Complete      
-----------------------------


In [20]:
#checking no of list items in the city_date
len(city_data)

694

In [21]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Weather
0,Norman Wells,65.2820,-126.8329,-23.80,76,75,1.99,CA,2021-02-07 20:12:35,broken clouds
1,Bluff,-46.6000,168.3333,56.21,79,96,5.21,NZ,2021-02-07 20:12:35,overcast clouds
2,Qaanaaq,77.4840,-69.3632,12.27,86,61,5.99,GL,2021-02-07 20:12:36,broken clouds
3,Bagdarin,54.4333,113.6000,-5.85,86,100,1.54,RU,2021-02-07 20:12:09,overcast clouds
4,Waipawa,-41.4122,175.5153,64.99,82,75,19.57,NZ,2021-02-07 20:12:36,broken clouds
5,Hobart,-42.8794,147.3294,55.40,62,40,13.80,AU,2021-02-07 20:10:18,scattered clouds
6,Dunedin,-45.8742,170.5036,60.01,89,94,3.00,NZ,2021-02-07 20:17:47,overcast clouds
7,Sao Filipe,14.8961,-24.4956,70.39,72,100,12.35,CV,2021-02-07 20:14:00,overcast clouds
8,Khatanga,71.9667,102.5000,-30.68,79,0,10.16,RU,2021-02-07 20:17:47,clear sky
9,Georgetown,5.4112,100.3354,80.60,83,20,3.44,MY,2021-02-07 20:17:47,few clouds


In [23]:
#reorder the columns as City, Country, Date, Lat, Lng, Max Temp, Humidity, Cloudiness, and Wind Speed,current description, so they are easy to read.
#Recall that to reorder the columns, we assign a variable to an array of the columns in the order we want them to appear:
new_column_order = ["City", "Country", "Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Current Weather"]
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,Norman Wells,CA,2021-02-07 20:12:35,65.2820,-126.8329,-23.80,76,75,1.99,broken clouds
1,Bluff,NZ,2021-02-07 20:12:35,-46.6000,168.3333,56.21,79,96,5.21,overcast clouds
2,Qaanaaq,GL,2021-02-07 20:12:36,77.4840,-69.3632,12.27,86,61,5.99,broken clouds
3,Bagdarin,RU,2021-02-07 20:12:09,54.4333,113.6000,-5.85,86,100,1.54,overcast clouds
4,Waipawa,NZ,2021-02-07 20:12:36,-41.4122,175.5153,64.99,82,75,19.57,broken clouds


In [24]:
#rename column name to Current Description as per the module requirement. 
city_data_df = city_data_df.rename(columns={"Current Weather": "Current Description"})
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Norman Wells,CA,2021-02-07 20:12:35,65.2820,-126.8329,-23.80,76,75,1.99,broken clouds
1,Bluff,NZ,2021-02-07 20:12:35,-46.6000,168.3333,56.21,79,96,5.21,overcast clouds
2,Qaanaaq,GL,2021-02-07 20:12:36,77.4840,-69.3632,12.27,86,61,5.99,broken clouds
3,Bagdarin,RU,2021-02-07 20:12:09,54.4333,113.6000,-5.85,86,100,1.54,overcast clouds
4,Waipawa,NZ,2021-02-07 20:12:36,-41.4122,175.5153,64.99,82,75,19.57,broken clouds


In [26]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv" #removed weather_database from the path as the code is already in this folder.
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")